In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os, sys
import spikeinterface.full as si
import h5py
import numpy as np
from tqdm import tqdm
from glob import glob
sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import spike_sorting as ss
from axon_tracking import template_extraction as te
from axon_tracking import utils as ut
import matplotlib.pyplot as plt

In [ ]:
te_params = dict()
te_params['n_jobs'] = 16 #Number of cores to use for waveform extraction
te_params['filter_band'] = 150 #Either float for the highpass filter frequency or list for the bandpass filter frequencies
te_params['overwrite'] = False #Flag if templates should be recalculated if already existing
te_params['max_spikes_per_unit'] = 1000 #Maximum number of spikes to be used for template extraction

qc_params = dict()
qc_params['min_n_spikes'] = 1500 #Minimum number of spikes to be detected for a unit for template extraction to take place
qc_params['exclude_mua'] = True #Exclude units that were labelled multi unit activity by kilosort
qc_params['use_bc'] = False #Use bombcell for QC
qc_params['use_si'] = True #Use spikeinterface for QC
qc_params["auto_merge"] = True #Automatically merge units (spikeinterface implementation)
qc_params["remove_redundant"] = True #Remove redundant units (spikeinterface implementation)

In [ ]:
sorting_list = ['/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/Slices/230711/Rec5/well000/sorter_output']

In [ ]:
te.extract_templates_from_sorting_list(sorting_list, qc_params=qc_params, te_params=te_params)

In [ ]:
sorting = si.KiloSortSortingExtractor(sorting_list[0])

In [ ]:
rec_path = ss.get_recording_path(sorting)

In [ ]:
from axon_tracking import visualization as vis #Separate as it is not needed for the template extraction

In [ ]:
root_path = sorting_path
plot_path = os.path.join(root_path, 'templates')

vis.plot_template_overview(plot_path,vmax=5,overwrite=True)

In [ ]:
test = np.load(
    '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/Slices/230711/Rec5/well000/sorter_output/templates/653.npy')

Try to reconstruct network activity from frame numbers of axon scan

In [ ]:
mxw = h5py.File(full_path)
fnos = mxw['recordings']['rec0000']['well005']['groups']['routed']['frame_nos']